## SD v1.5 Resource Lists
**Homepage :** [SD Webui Complete setup](https://github.com/ffxvs/sd-webui-complete-setup)  
**Guide :** [Resource Lists Guide](https://github.com/ffxvs/sd-webui-complete-setup/wiki/Resource-Lists-Guide)

### Check for Updates

**Current version : 2024.03.06**

In [ ]:
import ipywidgets as widgets
from IPython.utils import capture 
import requests

button = widgets.Button(description="Download now", button_style='success')
output = widgets.Output()

def on_button_clicked(b, url, version):
  with output:
    print('Downloading...')
    !wget -nv -O /notebooks/sd15_resource_lists_{version}.ipynb {url}
    print(f'sd15_resource_lists_{version}.ipynb downloaded in the root directory')

currentVersion = '2024.03.06'
updateURL = 'https://raw.githubusercontent.com/ffxvs/sd-webui-complete-setup/main/updates.json'
res = requests.get(updateURL)

if res.status_code == 200:
    for notebook in res.json()['resources']:
        if notebook['id'] == 'sd15':
            print('Current version : ' + currentVersion)
            print('Latest version  : ' + notebook['version'])
            if (notebook['version'] > currentVersion):
                print('\nThere is new version')
                button.on_click(lambda b: on_button_clicked(b, notebook['url'], notebook['version']))
                display(button, output)
                print('\nChangelog :')
                print(variant['changelog'])
            else:
                print('\nThis is the latest version')            
else:
    print('Failed to check for updates') 
    print('Response code :', res.status_code)

## 1. Requirements

### • Variables and Functions
**REQUIRED EVERY TIME YOU RUN THIS NOTEBOOK**

In [ ]:
# Civitai Token - Required to download models that require login
civitaiToken = ''


##################################### IMPORT #######################################

import os, re, requests
import ipywidgets as widgets
from IPython.utils import capture
from IPython.display import clear_output

############################ ENVIRONMENT VARIABLES ################################

os.environ['LD_PRELOAD'] = '/lib/x86_64-linux-gnu/libtcmalloc.so.4'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['PYTHONWARNINGS'] = 'ignore'
os.environ['PIP_ROOT_USER_ACTION'] = 'ignore'
os.environ['PIP_DISABLE_PIP_VERSION_CHECK'] = '1'

########################### GLOBAL PATHS AND FUNCTION ###########################

# Paths
root = '/notebooks'
a1111 = '/stable-diffusion-webui'
forge = '/stable-diffusion-webui-forge'

if os.path.exists(root + a1111):
    webui = root + a1111
elif os.path.exists(root + forge):
    webui = root + forge
else:
    raise Exception('Stable Diffusion Webui directory is not found')

controlNetModelsPath = webui + "/models/ControlNet"
embeddingsPath = webui + "/embeddings"
modelsPath = webui + "/models/Stable-diffusion"
loraPath = webui + "/models/Lora"
upscalerPath = webui + "/models/ESRGAN"
vaePath = webui + "/models/VAE"

# Resource URLs
mainRepoURL = 'https://raw.githubusercontent.com/ffxvs/sd-webui-complete-setup/main'
controlNetURL = mainRepoURL + '/res/sd15/sd15-controlnet.json'
t2iAdapterURL = mainRepoURL + '/res/sd15/sd15-t2i-adapter.json'
animeModelsURL = mainRepoURL + '/res/sd15/models/sd15-anime-models.json'
generalModelsURL = mainRepoURL + '/res/sd15/models/sd15-general-models.json'
realisticModelsURL = mainRepoURL + '/res/sd15/models/sd15-realistic-models.json'
womanModelsURL = mainRepoURL + '/res/sd15/models/sd15-woman-models.json'
builtinResourcesURL = mainRepoURL + '/res/sd15/sd15-builtin-resources.json'
loraListURL = mainRepoURL + '/res/sd15/sd15-lora.json'
embeddingListURL = mainRepoURL + '/res/sd15/sd15-embeddings.json'
upscalerListURL = mainRepoURL + '/res/upscaler.json'
vaeListURL = mainRepoURL + '/res/sd15/sd15-vae.json'

boolean = [False, True]
requestHeaders = {
    "Cache-Control": "no-cache, no-store, must-revalidate",
    "Pragma": "no-cache",
    "Expires": "0"
}

# Complete message
def completedMessage(): 
    completed = widgets.Button(description='Completed', button_style='success', icon='check')
    print('\n')
    display(completed)

# Resource
def getResource(url):
    res = requests.get(url, headers=requestHeaders)
    if res.status_code == 200:
        return res.json()
    else:
        return False

# Aria2c
def downloader(url, path, overwrite=False):
    args = '--download-result=hide --console-log-level=error -c -x 16 -s 16 -k 1M '
    if overwrite: args += '--allow-overwrite'
    if url.startswith('https://civitai.com/api/download/') and civitaiToken:
        if '?' in url:
            url = f'{url}&token={civitaiToken}'
        else:
            url = f'{url}?token={civitaiToken}'
    formattedURL = '"' + url + '"'
    if bool(re.search(r'\/[\w\.-]+\.\w+$', url)):
        filename = url.split('/')[-1]
        !aria2c {args} {formattedURL} -d {path} -o {filename}
    else:
        !aria2c {args} {formattedURL} -d {path}

# Git Clone
def silentClone(command, path, update=False, overwrite=False):
    directory = command.split('/')[-1]
    if os.path.exists(path + '/' + directory):
        if update:
            os.chdir(f'{path}/{directory}')
            !git pull -q
        elif overwrite:
            !rm -r {path}/{directory}
            !git clone -q --depth 10 {command} {path}/{directory}
    else:
        !git clone -q --depth 10 {command} {path}/{directory}

# WGet
def silentGet(command):
    !wget -nv {command}

# ControlNet
def downloadControlNet(controlNet, url):
    controlNetData = getResource(url)
    for model in controlNet:
        if controlNet[model]:
            print('\n' + model + '...')
            for url in controlNetData[model]:
                downloader(url, f'{controlNetModelsPath}/sd')
                print('')

# Model Mapper
def modelMapper(name, version, url):
    return {
        'name': name,
        'version': version,
        'url': url
    }

# Selected Models
def selectedModels(modelsURL):
    modelList = []
    for model in getResource(modelsURL)['models']:
        isSelected = eval(model['id'])
        if isSelected != 'Select version...':
            for variant in model['variants']:
                if variant['version'] == isSelected:
                    modelList.append(modelMapper(
                        model['name'],
                        variant['version'],
                        variant['url']
                    ))
    return modelList

# Download models
def downloadModels(modelsURL):
    print('⏳ Downloading selected models...')
    for model in selectedModels(modelsURL):
        print(f"\n* {model['name']} | {model['version']}")
        downloader(model['url'], f'{modelsPath}/sd')

# Built-in resources
def builtinResources(resourceList, resourcePath):
    for resource in resourceList:
        print(f"\n{resource['name']}...")
        downloader(resource['url'], resourcePath)

def downloadBuiltinResources():
    resource = getResource(builtinResourcesURL)
    print('⏳ Downloading built-in LoRA...')
    builtinResources(resource['lora'], f'{loraPath}/sd')
    print('\n\n⏳ Downloading built-in Embeddings...')
    for embedding in resource['embeddings']:
        print(embedding['name'] + '...')
        silentClone(embedding['url'], f'{embeddingsPath}/sd', True)
    print('\n\n⏳ Downloading built-in upscaler...')
    builtinResources(resource['upscaler'], upscalerPath)
    print('\n\n⏳ Downloading built-in VAE...')
    builtinResources(resource['vae'], f'{vaePath}/sd')

# Selected resources
def selectedResources(resourceList, resourcePath):
    for resource in resourceList:
        if eval(resource['id']):
            print(f"\n {resource['name']}...")
            for url in resource['url']:
                downloader(url, resourcePath)
                print('')

def downloadSelectedResources():
    print('\n\n⏳ Downloading selected LoRA...')
    selectedResources(getResource(loraListURL)['lora'], f'{loraPath}/sd')
    print('\n\n⏳ Downloading selected embeddings...')
    for embedding in getResource(embeddingListURL)['embeddings']:
        if eval(embedding['id']):
            print(embedding['name'] + '...')
            silentClone(embedding['url'], f'{embeddingsPath}/sd', True)
    print('\n\n⏳ Downloading selected upscaler...')
    selectedResources(getResource(upscalerListURL)['upscaler'], upscalerPath)
    print('\n\n⏳ Downloading selected VAE...')
    selectedResources(getResource(vaeListURL)['vae'], f'{vaePath}/sd')

# Other resources
def otherResources(resourceList, resourcePath):
    for resource in resourceList:
        print('\n' + resource)
        downloader(resource, resourcePath)

def downloadOtherResources(otherLora, otherEmbeddings, otherUpscaler, otherVAE):
    if otherLora:
        print('\n\n⏳ Downloading LoRA...')
        otherResources(otherLora, f'{loraPath}/sd')
    if otherEmbeddings:
        print('\n\n⏳ Downloading embeddings...')
        otherResources(otherEmbeddings, f'{embeddingsPath}/sd')
    if otherUpscaler:
        print('\n\n⏳ Downloading upscaler...')
        otherResources(otherUpscaler, upscalerPath)
    if otherVAE:
        print('\n\n⏳ Downloading VAE...')
        otherResources(otherVAE, f'{vaePath}/sd')

# Remove git from directory
def removeGit(directory):
    for (folder, subfolders, files) in os.walk(directory):
        if subfolders:
            for subfolder in subfolders:
                if subfolder == '.git':
                    pathFolder = os.path.join(folder, subfolder)
                    !rm -r {pathFolder}

        for file in files:
            if file == '.gitattributes' or file == 'readme.md' or file == 'README.md':
                pathFile = os.path.join(folder, file)
                !rm {pathFile}

completedMessage()

## 2. ControlNet

### • ControlNet Models

In [ ]:
# ControlNet - https://github.com/lllyasviel/ControlNet-v1-1-nightly
# Select models to download. Default models are : depth, ip-adapter, openpose, tile
controlNet = {
    'canny': boolean[0],
    'depth': boolean[1],
    'inpaint': boolean[0],
    'ip2p': boolean[0],
    'ipadapter': boolean[1],
    'lineart': boolean[0],
    'lineart_anime': boolean[0],
    'mlsd': boolean[0],
    'normal': boolean[0],
    'openpose': boolean[1],
    'recolor': boolean[0],
    'scribble': boolean[0],
    'seg': boolean[0],
    'shuffle': boolean[0],
    'softedge': boolean[0],
    'tile': boolean[1]
}

# T2I-Adapter - https://github.com/TencentARC/T2I-Adapter/tree/SD
# A lighter alternative to offical controlnet models. Select models to download. 
t2iAdapter = {  
    'canny': boolean[0],
    'color': boolean[0],
    'depth': boolean[0],
    'keypose': boolean[0],
    'openpose': boolean[0],
    'seg': boolean[0],
    'sketch': boolean[0],
    'style': boolean[0],
    'zoedepth': boolean[0]
}


################################################################################################################

print("\n⏳ Download selected controlNet models...")
downloadControlNet(controlNet, controlNetURL)
print("\n⏳ Download selected T2I-Adapter...")
downloadControlNet(t2iAdapter, t2iAdapterURL)
os.chdir(webui)
completedMessage()

### • Install from URLs

In [ ]:
# Install controlNet models from URL 
otherControlNet = []


########################################################################################

if otherControlNet:
    print('⏳ Downloading controlNet models...')
    otherResources(otherControlNet, f'{controlNetModelsPath}/sd')

os.chdir(webui)
completedMessage()

## 3. SD v1.5 Models
**Choose models you want to download**  
Some models hosted by Civitai require you to login to download.  
Create an API key in [Civitai user settings](https://civitai.com/user/account) then paste it to `civiaiToken` inside Variables and Functions cell.

### • Anime / Cartoon / 3D

In [ ]:
# Anything - https://civitai.com/models/9409
anythingVersions = ['Select version...', 'Ink']
anything = anythingVersions[0]

# Babes - https://civitai.com/models/2220
babesVersions = ['Select version...', 'v3']
babes = babesVersions[0]

# Cetus-Mix - https://civitai.com/models/6755
cetusMixVersions = ['Select version...', 'WhaleFall2']
cetusMix = cetusMixVersions[0]

# Counterfeit - https://civitai.com/models/4468
counterfeitVersions = ['Select version...', 'v3']
counterfeit = counterfeitVersions[0]

# MeinaMix - https://civitai.com/models/7240
meinaMixVersions = ['Select version...', 'v11', 'v11-inpainting']
meinaMix = meinaMixVersions[0]

# ReV Animated - https://civitai.com/models/7371
revAnimatedVersions = ['Select version...', 'v1.2.2']
revAnimated = revAnimatedVersions[0]

# ToonYou - https://civitai.com/models/30240
toonYouVersions = ['Select version...', 'beta 6']
toonYou = toonYouVersions[0]


########################################################################################

downloadModels(animeModelsURL)
completedMessage()

### • General Purpose

In [ ]:
# Deliberate - https://huggingface.co/XpucT/Deliberate
deliberateVersions = ['Select version...', 'v5', 'v5-inpainting']
deliberate = deliberateVersions[0]

# DreamShaper - https://civitai.com/models/4384
dreamShaperVersions = ['Select version...', 'v8', 'v8-inpainting']
dreamShaper = dreamShaperVersions[0]

# Experience - https://civitai.com/models/5952/experience
experienceVersions = ['Select version...', 'v10', 'Realistic Exp v3']
experience = experienceVersions[0]

# NeverEnding Dream (NED) - https://civitai.com/models/10028
nedVersions = ['Select version...', 'v1.22_vae', 'v1.22-inpainting']
ned = nedVersions[0]

# Protogen - https://civitai.com/user/darkstorm2150
protogenVersions = ['Select version...', 
    'x2.2 (Anime)', 
    'x3.4 (Photorealism)', 
    'x5.3 (Photorealism)', 
    'x5.8 (Sci-fi + Anime)'
]
protogen = protogenVersions[0]


########################################################################################

downloadModels(generalModelsURL)
completedMessage()

### • Realistic

In [ ]:
# Absolute Reality - https://civitai.com/models/81458
absoluteRealityVersions = ['Select version...', 'v1.8.1', 'v1.8.1-inpainting']
absoluteReality = absoluteRealityVersions[0]

# Analog Madness - https://civitai.com/models/8030
analogMadnessVersions = ['Select version...', 'v7', 'v7-inpainting']
analogMadness = analogMadnessVersions[0]

# CyberRealistic - https://civitai.com/models/15003
cyberRealisticVersions = ['Select version...', 'v4.2', 'v4.1-inpainting']
cyberRealistic = cyberRealisticVersions[0]

# epiCPhotoGasm - https://civitai.com/models/132632
epicPhotogasmVersions = ['Select version...', 'AmateurRealLife', 'Last Unicorn', 'Z-inpainting']
epicPhotogasm = epicPhotogasmVersions[0]

# epiCRealism - https://civitai.com/models/25694
epicRealismVersions = ['Select version...', 'Natural Sin', 'Pure Evo v5', 'Pure Evo v5-inpainting']
epicRealism = epicRealismVersions[0]

# Realistic Vision - https://civitai.com/models/4201
realVisionVersions = ['Select version...', 'v6_b1_vae', 'v6_b1_vae-inpainting']
realVision = realVisionVersions[0]

# Reliberate - https://huggingface.co/XpucT/Reliberate
reliberateVersions = ['Select version...', 'v3', 'v3-inpainting']
reliberate = reliberateVersions[0]


########################################################################################

downloadModels(realisticModelsURL)
completedMessage()

### • Woman

In [ ]:
# BRA (Beautiful Realistic Asians) - https://civitai.com/models/25494
braVersions = ['Select version...', 'v7']
bra = braVersions[0]

# BracingEvoMix - https://huggingface.co/sazyou-roukaku/BracingEvoMix
bemVersions = ['Select version...', 'v2']
bem = bemVersions[0]

# Chilled Remix - https://huggingface.co/sazyou-roukaku/chilled_remix
chilledRemixVersions = ['Select version...', 'v2', 'Reversemix v2']
chilledRemix = chilledRemixVersions[0]

# Henmix Real - https://civitai.com/models/20282
henmixRealVersions = ['Select version...', 'v4', 'v5c']
henmixReal = henmixRealVersions[0]

# majicMIX Realistic - https://civitai.com/models/43331
majicMixRealVersions = ['Select version...', 'v7', 'v7-inpainting']
majicMixReal = majicMixRealVersions[0]


########################################################################################

downloadModels(womanModelsURL)
completedMessage()

### • Install from URLs

In [ ]:
# Install models from URL 
otherModels = []


########################################################################################

if otherModels:
    print('⏳ Downloading models...')
    otherResources(otherModels, f'{modelsPath}/sd')

os.chdir(webui)
completedMessage()

## 4. LoRA, Embedding, Upscaler and VAE

### • Built-in Resources
**This is just for information. No need to run this cell.**

**Pre-installed LoRA :**
* [Add More Details - Detail Enhancer/Tweaker](https://civitai.com/models/82098)
* [Detail Tweaker](https://civitai.com/models/58390)
* [Epi Noise Offset](https://civitai.com/models/13941/epinoiseoffset)
* [LowRA](https://civitai.com/models/48139/lowra)

**Pre-installed Embeddings :**  
* [Embeddings Collection](https://huggingface.co/ffxvs/embeddings-collection)
* [Negative Prompts Pack](https://huggingface.co/ffxvs/negative-prompts-pack) 

**Pre-installed Upscaler :**  
* [UltraSharp](https://openmodeldb.info/models/4x-UltraSharp)

**Pre-installed VAE :**  
* [vae-ft-mse-840000](https://huggingface.co/stabilityai/sd-vae-ft-mse-original)

### • Resource Lists

In [ ]:
####################################### LORA ##########################################

# Depth of Field Slider - https://civitai.com/models/135380
dopSlider = boolean[0]

# Doll Likeness Series - https://huggingface.co/Kanbara/doll-likeness-series
dollLikeness = boolean[0]

# Hands SD v1.5 - https://civitai.com/models/200255?modelVersionId=228003
handsLora = boolean[0]

# Intricate Background & Background Blur - https://civitai.com/models/105732
bgDetails = boolean[0]

# LCM Lora SD v1.5 - https://huggingface.co/latent-consistency/lcm-lora-sdv1-5
lcmLora = boolean[0]


############################ EMBEDDINGS (TEXTUAL INVERSION) ############################

# SCG Emotions Pack - https://civitai.com/models/8860
scgEmotion = boolean[0]

# SCG Embedding Toolkit - https://civitai.com/models/8551
scgToolkit = boolean[0]

# Zovya Age Slider - https://civitai.com/models/65214
ageSlider = boolean[0]

# Zovya Gender Slider - https://civitai.com/models/89709
genderSlider = boolean[0]


###################################### UPSCALER ########################################

# BSRGAN - https://openmodeldb.info/models/4x-BSRGAN
bsrgan = boolean[0]

# LSDIRPlus - https://openmodeldb.info/models/4x-LSDIRplus
lsdirPlus = boolean[0]

# NMKD Superscale - https://openmodeldb.info/models/4x-NMKD-Superscale
superscale = boolean[0]

# Nomos8kSC - https://openmodeldb.info/models/4x-Nomos8kSC
nomos8ksc = boolean[0]

# Remacri - https://openmodeldb.info/models/4x-Remacri
remacri = boolean[0]

# UltraMix - https://upscale.wiki/w/index.php?title=Model_Database&oldid=1571
ultraMix = boolean[0]

# Valar - https://openmodeldb.info/models/4x-Valar
valar = boolean[0]


######################################## VAE ############################################

# Blessed2 - https://civitai.com/models/118561?modelVersionId=142467
blessed2 = boolean[0]

# ClearVAE - https://civitai.com/models/22354
clearVAE = boolean[0]

# kl-f8-anime2 - https://civitai.com/models/23906
kl_f8_anime2 = boolean[0]

# NAI/NovelAI - https://huggingface.co/WarriorMama777/OrangeMixs
novelAI = boolean[0]


###################################################################################################

os.chdir(webui)
downloadBuiltinResources()
downloadSelectedResources()
os.chdir(webui)
completedMessage()

### • Install from URLs

In [ ]:
# LoRA
otherLora = []

# Embeddings
otherEmbeddings = []

# Upscaler
otherUpscaler = []

# VAE
otherVAE = []


###################################################################################################

downloadOtherResources(otherLora, otherEmbeddings, otherUpscaler, otherVAE)
os.chdir(webui)
completedMessage()